In [4]:
# !pip install -U encodec -i https://mirrors.aliyun.com/pypi/simple/
# !pip install -U noisereduce -i https://mirrors.aliyun.com/pypi/simple/
# !pip install -U git+https://git@github.com/facebookresearch/encodec#egg=encodec  # bleeding edge 

In [5]:
import warnings
warnings.filterwarnings("ignore")

from encodec import EncodecModel
from encodec.utils import convert_audio

import torchaudio
import torch
import glob
import os
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
import json

import sys
sys.path.append("../sho_util/pyfiles/")
from sound import play_audio

# Preparation

In [6]:
dataset_dir = "../../Dataset/Blizzard2013/"
wav_dir = dataset_dir + "BC2013_segmented_v1_wav_selection/"
txt_dir = dataset_dir + "BC2013_segmented_v1_txt_selection/"
pho_dir = dataset_dir + "BC2013_segmented_v1_phonemes_selection/"
enc_dir = dataset_dir + "BC2013_segmented_v1_encodec_selection/"
enc_dim = 1024
nq = 1

wavlist = {}
txtlist = {}
pholist = {}
enclist = {}
wavlist["train"] = glob.glob(wav_dir+"C*/*.wav")
wavlist["train"].sort()
txtlist["train"] = glob.glob(txt_dir+"C*/*.txt")
txtlist["train"].sort()
pholist["train"] = glob.glob(pho_dir+"C*/*.npy")
pholist["train"].sort()
enclist["train"] = glob.glob(enc_dir+"C*/*.npy")
enclist["train"].sort()

wavlist["test"] = glob.glob(wav_dir+"te*/*.wav")
wavlist["test"].sort()
txtlist["test"] = glob.glob(txt_dir+"te*/*.txt")
txtlist["test"].sort()
pholist["test"] = glob.glob(pho_dir+"te*/*.npy")
pholist["test"].sort()
enclist["test"] = glob.glob(enc_dir+"te*/*.npy")
enclist["test"].sort()

In [7]:
for mode in ["train", "test"]:
    train_data = []
    num = len(txtlist[mode])
    num = min(500, len(txtlist[mode]))
    for idx in tqdm(range(num)):
        f = open(txtlist[mode][idx], "r")
        text = f.read()
        phonemes = np.load(pholist[mode][idx])
        encodec = np.load(enclist[mode][idx])[0,:nq]
        data = {}
        data["text"] = text
        
        # phonemes_str = "".join([f"[{str(t)}]" for t in phonemes])
        phonemes_str = "".join([f"<{str(t)}>" for t in phonemes])
        data["phonemes"] = "<PHN>" + phonemes_str + "</PHN>"
        
        
        recoded = encodec + np.arange(encodec.shape[0]).reshape(-1,1)*enc_dim
        # code_str = "".join([f"[{str(t)}]" for t in recoded.T.reshape(-1)])
        code_str = "".join([f"<{str(t)}>" for t in recoded.T.reshape(-1)])
        data["output"] = "<SPCH>" + code_str + "</SPCH>"
        
        train_data.append(data)
        
    with open(f"./datasets/tts_data_{mode}_{nq}_small.json", 'w') as f:
        json.dump(train_data, f, indent=4)

100%|██████████| 200/200 [00:04<00:00, 45.71it/s]


---

In [70]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["text"],
        data_point["phonemes"],
        data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    if not train_on_inputs:
        user_prompt = prompter.generate_prompt(
            data_point["text"], data_point["phonemes"]
        )
        tokenized_user_prompt = tokenize(
            user_prompt, add_eos_token=add_eos_token
        )
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        tokenized_full_prompt["labels"] = [
            -100
        ] * user_prompt_len + tokenized_full_prompt["labels"][
            user_prompt_len:
        ]  # could be sped up, probably
    return tokenized_full_prompt

In [67]:
data_path = "test.json"
data = load_dataset("json", data_files=data_path)
data_point = data["train"][0]
a = prompter.generate_prompt(
    data_point["text"],
    data_point["phonemes"],
    data_point["output"],
)

In [58]:
template = "alpaca_tts"
prompter = Prompter_TTS(template)

In [62]:
if val_set_size > 0:
    train_val = data["train"].train_test_split(
        test_size=val_set_size, shuffle=True, seed=42
    )
    train_data = (
        train_val["train"].shuffle().map(generate_and_tokenize_prompt)
    )
    val_data = (
        train_val["test"].shuffle().map(generate_and_tokenize_prompt)
    )
else:
    train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
    val_data = None

Map: 100%|██████████| 3/3 [00:00<00:00, 46.49 examples/s]


In [ ]:
ge

In [64]:
train_data[0]

{'output': '<865> <1448> <2985> <3988> <738> <1883> <2869> <4028> <475> <1993> <2941> <3988> <747> <1809> <2607> <3758> <531> <1240> <2738> <4058> <679> <1702> <2943> <3456> <953> <1209> <2543> <3712> <651> <1641> <2341> <3784> <224> <1955> <2703> <3837> <136> <1641> <2918> <3146> <432> <1956> <3018> <4091> <677> <1627> <3050> <3744> <879> <1126> <2753> <3565> <310> <1153> <2260> <4070> <724> <1290> <2666> <3805> <224> <1419> <2800> <3961> <30> <1679> <2703> <3436> <155> <1955> <2342> <3871> <20> <1968> <2085> <3554> <683> <1238> <2620> <3397> <230> <1643> <2118> <3276> <695> <1748> <2796> <3538> <501> <1258> <2169> <3346> <679> <1617> <2465> <4053> <967> <1419> <2796> <3914> <408> <1753> <2982> <3910> <62> <1993> <3048> <3600> <408> <1568> <2985> <4028> <475> <1604> <2695> <3852> <537> <1240> <2462> <3515> <103> <1672> <2275> <3834> <25> <1865> <2589> <3398> <751> <1747> <2923> <3484> <502> <1235> <2700> <4058> <136> <1125> <2646> <3217> <321> <1988> <3028> <3394> <143> <1482> <2780> 